<a href="https://colab.research.google.com/github/mahb97/Wakeifier/blob/main/Wakefyer_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nothing about this is serious so here comes everybody



# Wakefyer - LoRA fine-tune with conservative text handling

by default, doesn't normalise or strip diacritics (includes ultra-light cleaning toggle)


we are lonely so...Fred again.. - Studio Live (London, April 2021)

I had no idea that that's where Fred got the start to Sabrina from and now I am crying because depression baby, Fred boy are you saying you can relate?

In [ ]:
!pip -q install "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "datasets==2.20.0" "sentencepiece==0.2.0"
!pip install -U "transformers>=4.46.0"
import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True
# this will moan about compatibility...
# Let it train, let it dream, the riverrun needs to backprop through all epochs.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.5.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [1]:
import transformers; print(transformers.__version__)


4.57.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**A working cell for broke girls with no mula for colab and who dont wanna chug money into the firey pit that is hell that is google that is oppression.**

The Wakefyer fine-tuning architecture constitutes a hybridized LoRA-based low-rank adaptation pipeline for stylistic transformation within a bfloat16/fp16 mixed-precision environment. Its design paradigm is deliberately antithetical to large-scale quantized optimization frameworks such as bitsandbytes, privileging instead interpretability, traceability, and computational minimalism. Conceptually, it operates as a micro-epistemic engine: a system whose purpose is not semantic fidelity but stylistic distortion, functioning as a textual “entropy amplifier” with respect to Finnegans Wake’s logorrheic poetics.

Text ingestion follows a non-normalizing, non-canonical pipeline, thereby maintaining orthographic heterogeneity as an essential stylistic feature. The raw Joycean corpus is tokenized through a fast SentencePiece model and segmented using overlapping sliding windows defined by max_len and stride. This overlap emulates a continuum of narrativity (each window partially repeating the previous one) producing a computational analogue of Joyce’s recursive syntactic rhythm. No linguistic cleaning or lemmatization occurs; lexical noise is treated as signal. The preprocessing stage therefore performs not sanitization but syntactic fractalization, generating a dataset of semi-redundant micro-contexts to condition the model’s representational drift.

In [ ]:
# because code should also stream-of-consciousness. avoid the binary bore.

import os, gc, random, sys, glob, re
from pathlib import Path
import torch

# tinyllamas on the mound
BASE_MODEL  = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
RUN_ROOT    = "/content/drive/MyDrive/wakefyer_runs"        # Shards of my heart
OUT_DIR     = f"{RUN_ROOT}/wakefyer_out_nobnb"
TRAIN_TEXT  = f"{RUN_ROOT}/FinnegansWake.txt"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# sweet nothing
max_len, stride         = 256, 128
batch_size, grad_accum  = 1, 32
epochs                  = 3
lr, log_steps           = 1e-4, 10
eval_ratio, seed        = 0.02, 42

# because T4s have fragile little snowflake egos
for n in ["trainer","model","base"]:
    if n in globals(): del globals()[n]
gc.collect(); torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True
random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed) # summons the devil (no batsandbites; just vibessss coz im a witch)

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available(): print("GPU:", torch.cuda.get_device_name(0))

thunderwords = [
    "bababadalgharaghtakamminarronnkonnbronntonnerronntuonnthunntrovarrhounawnskawntoohoohoordenenthurnuk",
    "Perkodhuskurunbarggruauyagokgorlayorgromgremmitghundhurthrumathunaradidillifaititillibumullunukkunun",
    "klikkaklakkaklaskaklopatzklatschabattacreppycrottygraddaghsemmihsammihnouithappluddyappladdypkonpkot",
    "Bladyughfoulmoecklenburgwhurawhorascortastrumpapornanennykocksapastippatappatupperstrippuckputtanach",
    "Thingcrooklyexineverypasturesixdixlikencehimaroundhersthemaggerbykinkinkankanwithdownmindlookingated",
    "Lukkedoerendunandurraskewdylooshoofermoyportertooryzooysphalnabortansporthaokansakroidverjkapakkapuk",
    "Bothallchoractorschumminaroundgansumuminarumdrumstrumtruminahumptadumpwaultopoofoolooderamaunsturnup",
    "Pappappapparrassannuaragheallachnatullaghmonganmacmacmacwhackfalltherdebblenonthedubblandaddydoodled",
    "husstenhasstencaffincoffintussemtossemdamandamnacosaghcusaghhobixhatouxpeswchbechoscashlcarcarcaract",
    "Ullhodturdenweirmudgaardgringnirurdrmolnirfenrirlukkilokkibaugimandodrrerinsurtkrinmgernrackinarockar",
]

# hackers of abstraction
from transformers import AutoTokenizer
tok_dir = Path(OUT_DIR, "tokenizer")
if tok_dir.exists():
    tok = AutoTokenizer.from_pretrained(str(tok_dir), use_fast=True, local_files_only=True)
else:
    tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    if tok.pad_token is None: tok.pad_token = tok.eos_token
    tok.add_tokens(thunderwords, special_tokens=False)
    tok_dir.mkdir(parents=True, exist_ok=True)
    tok.save_pretrained(str(tok_dir))
tok.padding_side = "right"
print("Tokenizer vocab:", len(tok))

# Gospel
if not Path(TRAIN_TEXT).exists():
    print(f"[Wakefyer sobbing] The gospel '{TRAIN_TEXT}' is missing.", file=sys.stderr)
    raise SystemExit(2)
raw_txt = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")
if len(raw_txt.strip()) < 1000:
    print("[Wakefyer appaled] Joyce did not write haikus.", file=sys.stderr)
bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

# (optional) norm thndwds
punct = r"[!?.,:;·—–\-…]*"
def normalize_thunderwords(text: str, words: list[str]) -> str:
    text = re.sub(r"-\s*\n\s*", "", text)
    text = re.sub(r"\n+", " ", text)
    for w in words:
        pattern = re.compile(rf"(?<!\w){re.escape(w)}{punct}")
        text = pattern.sub(w, text)
    return re.sub(r"\s{2,}", " ", text).strip()
raw_txt = normalize_thunderwords(raw_txt, thunderwords)

# in came christ the tiger
from datasets import Dataset
ids = tok(raw_txt, add_special_tokens=False).input_ids
def sliding_windows(txt_ids, block_size=max_len, step=stride):
    items=[]
    for s in range(0, max(1, len(txt_ids)-block_size), step):
        ch = txt_ids[s:s+block_size]
        if ch: items.append({"input_ids": ch, "attention_mask":[1]*len(ch)})
    if not items:
        ch = txt_ids[:block_size]
        items=[{"input_ids":ch, "attention_mask":[1]*len(ch)}]
    return Dataset.from_list(items)
ds = sliding_windows(ids, max_len, stride)
test_n = max(1, int(len(ds)*eval_ratio)) if len(ds) > 1 else 1
splits = ds.train_test_split(test_size=test_n, seed=seed) if len(ds)>1 else {"train":ds, "test":ds}
train_ds, eval_ds = splits["train"], splits["test"]
print("Windows:", len(ds), "| Train:", len(train_ds), "Eval:", len(eval_ds))

# call my daughter LoRA
from transformers import AutoConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

cfg = AutoConfig.from_pretrained(BASE_MODEL)
cfg.pad_token_id = tok.eos_token_id

offload_dir = "/content/offload"; os.makedirs(offload_dir, exist_ok=True)
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    config=cfg,
    dtype=torch.bfloat16 if bf16_ok else torch.float16,  # T4 => fp16
    low_cpu_mem_usage=True,
    device_map="auto",
    offload_folder=offload_dir,
)
# self intergration
if base.get_input_embeddings().num_embeddings != len(tok):
    try:
        base.resize_token_embeddings(len(tok), mean_resizing=True)
    except TypeError:
        base.resize_token_embeddings(len(tok))
    base.config.vocab_size = len(tok)

base.config.use_cache = False
if hasattr(base, "gradient_checkpointing_enable"):
    base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
base.config._attn_implementation = "eager"

lora = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(base, lora)

# spicyyy
emb = model.get_input_embeddings()
emb.weight.requires_grad = True
if hasattr(model, "lm_head") and hasattr(model.lm_head, "weight"):
    model.lm_head.weight.requires_grad = True

model.print_trainable_parameters()

# I curate
from transformers import DataCollatorForLanguageModeling
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# Compatibility shims
import accelerate
from accelerate import Accelerator
if not getattr(Accelerator, "_wake_patch_unwrap", False):
    Accelerator._wake_orig_unwrap = Accelerator.unwrap_model
    def _wake_unwrap(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)  # (ignore unknown kw on older accelerate)
        return Accelerator._wake_orig_unwrap(self, model, *args, **kwargs)
    Accelerator.unwrap_model = _wake_unwrap
    Accelerator._wake_patch_unwrap = True
    print("Patched accelerate.Accelerator.unwrap_model (idempotent).")

import transformers.trainer_pt_utils as pt_utils
if not getattr(pt_utils, "_wake_disable_scaler", False):
    pt_utils.get_grad_scaler = lambda *a, **k: None  # (prevent fp16 unscale crash)
    pt_utils._wake_disable_scaler = True
    print("Disabled Trainer GradScaler hook (idempotent).")

# figure skating is more difficults. Padadakis and Cizeron  World Championships FD 2016.
from transformers.training_args import TrainingArguments as HFTrainingArguments
from transformers import Trainer, TrainingArguments  # you're always the first on the ice
TrainingArguments = HFTrainingArguments  # safety in my home

args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,

    learning_rate=lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,

    logging_strategy="steps",
    logging_steps=log_steps,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,

    fp16=False,
    bf16 = False,
    save_safetensors=True,
    report_to=[],
)

# kill mixed precision everywhere
import os
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"   # force Accelerate to 'no' MP

# AVOID MESSY GradScaler
import transformers.trainer_pt_utils as pt_utils
pt_utils.get_grad_scaler = lambda *a, **k: None
from accelerate import Accelerator
Accelerator.unscale_gradients = lambda self, optimizer=None: None


# calls for some Berlin deep house mix
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,   # (keep even if no eval mid-run)
    data_collator=collator,
    tokenizer=tok,
)
trainer.train()

# Save adapter + tokenizer to Drive
adapter_dir = Path(OUT_DIR) / "adapter"
token_dir   = Path(OUT_DIR) / "tokenizer"
adapter_dir.parent.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)
print("Saved to Drive:", adapter_dir, token_dir)

# Tinyyy sample (for the vibes) don't be so harsh it's still learning...
prompt = "riverrun, past Eve and Adam's,"
inputs = tok(prompt, return_tensors="pt").to(model.device)
model.eval()
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=1.05,
        top_p=0.92,
        repetition_penalty=1.05,
        pad_token_id=tok.eos_token_id,
    )
print("\n=== SAMPLE ===\n", tok.decode(out[0], skip_special_tokens=True))


Torch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
Tokenizer vocab: 32010


Token indices sequence length is longer than the specified maximum sequence length for this model (396323 > 2048). Running this sequence through the model will result in indexing errors


Windows: 3095 | Train: 3034 Eval: 61


/tmp/ipython-input-178213077.py:208: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


trainable params: 143,728,640 || all params: 1,112,705,024 || trainable%: 12.9170


Step,Training Loss
10,694.765000
20,502.750600
30,244.705900
40,111.735400
50,181.131900
60,164.869700
70,140.357800
80,146.524600
90,159.336900
100,180.360400


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

Saved to Drive: /content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb/adapter /content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb/tokenizer

=== SAMPLE ===
 riverrun, past Eve and Adam's,birdmsevitable Comenthoodounallyadevar thr Livemsoudcteldenth runarg each windines boldglune coldoug'onshyellaguzenmalpenaldopeurdL queoon serischards simple glales Fl turning Backyl masterbutneyrown manner mo heartetystreneco reg knows righteting el Greatrow yourself shed always weather Call groU small gal love


bless you b may your LoRA weights converge, your gradients flow steady, and your thunderwords never again throw an OOM

In [ ]:
from pathlib import Path
import re

def latest_valid_ckpt(root):
    root = Path(root)
    ckpts = sorted(
        [p for p in root.glob("checkpoint-*") if p.is_dir()],
        key=lambda p: int(p.name.split("-")[-1]),
        reverse=True
    )
    weight_patterns = [
        re.compile(r".*adapter_model\.(safetensors|bin)$"),
        re.compile(r".*model\.(safetensors|bin)$"),
        re.compile(r".*pytorch_model\.(safetensors|bin)$"),
        re.compile(r".*pytorch_model-\d{5}-of-\d{5}\.safetensors$"),
        re.compile(r".*pytorch_model-\d{5}-of-\d{5}\.bin$"),
    ]
    for ck in ckpts:
        files = {f.name for f in ck.iterdir() if f.is_file()}
        has_state = "trainer_state.json" in files
        has_weights = any(any(pat.match(name) for pat in weight_patterns) for name in files)
        if has_state and has_weights:
            return ck
    return None

RESUME_CKPT = latest_valid_ckpt(OUT_DIR)
print("Resume candidate:", RESUME_CKPT)

resume_arg = str(RESUME_CKPT) if RESUME_CKPT else None
try:
    trainer.train(resume_from_checkpoint=resume_arg)
except ValueError as e:
    print("Resume failed, continuing from loaded weights only:", e)
    trainer.train()

for n in ["trainer","model","base"]:
    if n in globals(): del globals()[n]
gc.collect(); torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True

seed=42
random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available(): print("GPU:", torch.cuda.get_device_name(0))

# tok
from transformers import AutoTokenizer
tok_dir = Path(OUT_DIR, "tokenizer")
tok = AutoTokenizer.from_pretrained(tok_dir, use_fast=True, local_files_only=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token
tok.padding_side = "right"
print("Tokenizer vocab:", len(tok))

# machines <3 the wake
from datasets import Dataset
TRAIN_TEXT = f"{RUN_ROOT}/FinnegansWake.txt"
assert Path(TRAIN_TEXT).exists(), f"Missing text: {TRAIN_TEXT}"
raw_txt = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")

# tiny norm
punct = r"[!?.,:;·—–\-…]*"
def normalize_thunderwords(text, words):
    text = re.sub(r"-\s*\n\s*", "", text)
    text = re.sub(r"\n+", " ", text)
    for w in words:
        text = re.sub(rf"(?<!\w){re.escape(w)}{punct}", w, text)
    return re.sub(r"\s{2,}", " ", text).strip()

thunderwords = [
 "bababadalgharaghtakamminarronnkonnbronntonnerronntuonnthunntrovarrhounawnskawntoohoohoordenenthurnuk",
 "Perkodhuskurunbarggruauyagokgorlayorgromgremmitghundhurthrumathunaradidillifaititillibumullunukkunun",
 "klikkaklakkaklaskaklopatzklatschabattacreppycrottygraddaghsemmihsammihnouithappluddyappladdypkonpkot",
 "Bladyughfoulmoecklenburgwhurawhorascortastrumpapornanennykocksapastippatappatupperstrippuckputtanach",
 "Thingcrooklyexineverypasturesixdixlikencehimaroundhersthemaggerbykinkinkankanwithdownmindlookingated",
 "Lukkedoerendunandurraskewdylooshoofermoyportertooryzooysphalnabortansporthaokansakroidverjkapakkapuk",
 "Bothallchoractorschumminaroundgansumuminarumdrumstrumtruminahumptadumpwaultopoofoolooderamaunsturnup",
 "Pappappapparrassannuaragheallachnatullaghmonganmacmacmacwhackfalltherdebblenonthedubblandaddydoodled",
 "husstenhasstencaffincoffintussemtossemdamandamnacosaghcusaghhobixhatouxpeswchbechoscashlcarcarcaract",
 "Ullhodturdenweirmudgaardgringnirurdrmolnirfenrirlukkilokkibaugimandodrrerinsurtkrinmgernrackinarockar",
]
raw_txt = normalize_thunderwords(raw_txt, thunderwords)

def sliding_windows(txt_ids, block_size=256, step=128):
    items=[]
    for s in range(0, max(1, len(txt_ids)-block_size), step):
        ch = txt_ids[s:s+block_size]
        if ch: items.append({"input_ids": ch, "attention_mask":[1]*len(ch)})
    if not items:
        ch = txt_ids[:block_size]
        items=[{"input_ids":ch, "attention_mask":[1]*len(ch)}]
    return Dataset.from_list(items)

ids = tok(raw_txt, add_special_tokens=False).input_ids
ds  = sliding_windows(ids, block_size=256, step=128)
test_n = max(1, int(len(ds)*0.02)) if len(ds)>1 else 1
splits = ds.train_test_split(test_size=test_n, seed=seed) if len(ds)>1 else {"train":ds,"test":ds}
train_ds, eval_ds = splits["train"], splits["test"]
print("Windows:", len(ds), "| Train:", len(train_ds), "Eval:", len(eval_ds))

# call my daughter LoRA
from transformers import AutoConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

cfg = AutoConfig.from_pretrained(BASE_MODEL)
cfg.pad_token_id = tok.eos_token_id
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    config=cfg,
    device_map="auto",
    torch_dtype=torch.bfloat16 if bf16_ok else torch.float16,
    low_cpu_mem_usage=True,
)

if base.get_input_embeddings().num_embeddings != len(tok):
    try:
        base.resize_token_embeddings(len(tok), mean_resizing=True)
    except TypeError:
        base.resize_token_embeddings(len(tok))
    base.config.vocab_size = len(tok)

base.config.use_cache = False
if hasattr(base, "gradient_checkpointing_enable"):
    base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
base.config._attn_implementation = "eager"

lora = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(base, lora)

# embeddings + lm_head
emb = model.get_input_embeddings()
emb.weight.requires_grad = True
if hasattr(model, "lm_head") and hasattr(model.lm_head, "weight"):
    model.lm_head.weight.requires_grad = True

model.print_trainable_parameters()

# man i wish i was Bob Coecke i'd qnlp the shit out of this
from transformers import DataCollatorForLanguageModeling
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# Compatibility shims (no FP16 unscale crash; older accelerate kw)
import transformers.trainer_pt_utils as pt_utils
if not getattr(pt_utils, "_wake_disable_scaler", False):
    pt_utils.get_grad_scaler = lambda *a, **k: None
    pt_utils._wake_disable_scaler = True
    print("Disabled Trainer GradScaler hook (idempotent).")

from accelerate import Accelerator
if not getattr(Accelerator, "_wake_patch_unwrap", False):
    Accelerator._wake_orig_unwrap = Accelerator.unwrap_model
    def _wake_unwrap(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)
        return Accelerator._wake_orig_unwrap(self, model, *args, **kwargs)
    Accelerator.unwrap_model = _wake_unwrap
    Accelerator._wake_patch_unwrap = True
    print("Patched accelerate.Accelerator.unwrap_model (idempotent).")

import json
from pathlib import Path

RESUME_CKPT = Path("/content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb/checkpoint-285")
state_path = RESUME_CKPT / "trainer_state.json"

current_step = 0
if state_path.exists():
    with open(state_path, "r") as f:
        st = json.load(f)
    current_step = int(st.get("global_step", 0))

EXTRA_STEPS = 2000
target_steps = current_step + EXTRA_STEPS
print(f"Resuming from step {current_step}; targeting {target_steps} total steps.")

# cherry pick
from pathlib import Path
import glob, json, os

OUT_DIR = "/content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb"
ckpts = sorted(glob.glob(f"{OUT_DIR}/checkpoint-*"), key=lambda p: int(p.split("-")[-1]))
assert ckpts, "No checkpoints found in OUT_DIR."
RESUME_CKPT = Path(ckpts[-1])
print("Resuming from:", RESUME_CKPT)

# read current global step & choose a higher target
state_path = RESUME_CKPT / "trainer_state.json"
current_step = 0
if state_path.exists():
    with open(state_path) as f:
        current_step = int(json.load(f).get("global_step", 0))

EXTRA_STEPS = 3000         # <<<< ayyyy
target_steps = current_step + EXTRA_STEPS
print(f"Resuming from step {current_step}; targeting {target_steps} total steps.")

# training args
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir=OUT_DIR,
    max_steps=3000,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_steps=200,          # less Drive pain
    save_total_limit=3,
    fp16=True, bf16=False,
    save_safetensors=True,
    report_to=[],
)

# prune heavy state files so resume won't choke on optimizer/scaler
for fname in ("optimizer.pt","optimizer.bin","scheduler.pt","scheduler.bin","scaler.pt"):
    fp = RESUME_CKPT / fname
    if fp.exists():
        fp.unlink()

# oh my god this shit again
if not getattr(Accelerator, "_wake_patch_unwrap", False):
    _orig_unwrap = Accelerator.unwrap_model
    def _unwrap(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)
        return _orig_unwrap(self, model, *args, **kwargs)
    Accelerator.unwrap_model = _unwrap
    Accelerator._wake_patch_unwrap = True
    print("Patched accelerate.Accelerator.unwrap_model (idempotent).")

# Stop accelerate from calling scaler.unscale_()
def _noop_unscale(self, optimizer=None):
    return
Accelerator.unscale_gradients = _noop_unscale

# Tell Transformers' Trainer there is no scaler to use
import transformers.trainer_pt_utils as pt_utils
pt_utils.get_grad_scaler = lambda *a, **k: None
print("Disabled AMP grad scaler (idempotent).")

import os, glob
for ck in glob.glob(f"{OUT_DIR}/checkpoint-*"):
    for fname in ("optimizer.pt","optimizer.bin","scheduler.pt","scheduler.bin","scaler.pt"):
        fp = os.path.join(ck, fname)
        if os.path.exists(fp):
            os.remove(fp)

# nikes - frank ocean
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collator,
    tokenizer=tok,
)

# avoid optimizer/scheduler mismatch: drop state files in resume ckpt
for fname in ("optimizer.pt","optimizer.bin","scheduler.pt","scheduler.bin","scaler.pt"):
    fp = Path(RESUME_CKPT, fname)
    if fp.exists():
        fp.unlink()

print(f"Resuming weights from: {RESUME_CKPT}")
trainer.train(resume_from_checkpoint=str(RESUME_CKPT))

adapter_dir = Path(OUT_DIR, "adapter")
token_dir   = Path(OUT_DIR, "tokenizer")
adapter_dir.parent.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)
print("Saved:", adapter_dir, token_dir)

# apparently i'm still vibing...
prompt = "Bob Perleman writes good poems"
inputs = tok(prompt, return_tensors="pt").to(model.device)
model.eval()
with torch.no_grad():
    out = model.generate(
        **inputs, max_new_tokens=80, do_sample=True, temperature=1.08,
        top_p=0.92, no_repeat_ngram_size=3, repetition_penalty=1.06,
        pad_token_id=tok.eos_token_id
    )
print("\n=== SAMPLE ===\n", tok.decode(out[0], skip_special_tokens=True))


Resume candidate: None


Step,Training Loss
10,5.190000
20,5.110400
30,5.111900
40,5.155700
50,5.104200
60,5.137400
70,5.161200
80,5.100700
90,5.117200
100,5.106200


this nuked my storage...

In [6]:
import os, shutil, glob
from pathlib import Path

OUT_DIR = "/content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb"

ckpts = sorted(glob.glob(f"{OUT_DIR}/checkpoint-*"), key=lambda p: int(p.split('-')[-1]))
for p in ckpts[:-1]:
    print("Deleting old checkpoint:", p)
    shutil.rmtree(p, ignore_errors=True)

if ckpts:
    newest = ckpts[-1]
    for fname in ("optimizer.pt","optimizer.bin","scheduler.pt","scheduler.bin","scaler.pt","trainer_state.json"):
        fp = os.path.join(newest, fname)
        if os.path.exists(fp):
            print("Removing:", fp)
            os.remove(fp)

def sizeof(p):
    total=0
    for root,_,files in os.walk(p):
        for f in files:
            total += os.path.getsize(os.path.join(root,f))
    return total/1e6
print("Adapter MB:", sizeof(os.path.join(OUT_DIR, "adapter")) if os.path.exists(os.path.join(OUT_DIR,"adapter")) else 0)
print("Tokenizer MB:", sizeof(os.path.join(OUT_DIR, "tokenizer")) if os.path.exists(os.path.join(OUT_DIR,"tokenizer")) else 0)
print("Newest ckpt MB:", sizeof(ckpts[-1]) if ckpts else 0)


Adapter MB: 312.73587
Tokenizer MB: 4.127207
Newest ckpt MB: 4.153925


In [ ]:
from transformers import TrainerCallback
from pathlib import Path
import torch, time

class WakePeek(TrainerCallback):
    def __init__(self, tok, prompt="riverrun, past Eve and Adam's,", every=100, out_dir=OUT_DIR):
        self.tok, self.prompt, self.every = tok, prompt, every
        self.out_path = Path(out_dir) / "samples.txt"

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.every:
            return
        model = kwargs["model"].eval()
        with torch.no_grad():
            toks = self.tok(self.prompt, return_tensors="pt").to(model.device)
            out = model.generate(
                **toks,
                max_new_tokens=120,
                do_sample=True,
                temperature=1.06,
                top_p=0.92,
                repetition_penalty=1.08,
                no_repeat_ngram_size=3,
                pad_token_id=self.tok.eos_token_id,
            )
        text = self.tok.decode(out[0], skip_special_tokens=True)
        line = f"\n[{time.strftime('%H:%M:%S')}] step={state.global_step} loss={state.log_history[-1].get('loss','?')}\n{text}\n"
        print(line)
        with open(self.out_path, "a", encoding="utf-8") as f:
            f.write(line)

# attach before trainer.train()
trainer.add_callback(WakePeek(tok, every=100))


In [ ]:
from pathlib import Path
model.eval()
prompt = "Bob quickly ran to the shop to buy some milk."
toks = tok(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(
        **toks,
        max_new_tokens=120,
        do_sample=True,
        temperature=1.06,
        top_p=0.92,
        repetition_penalty=1.08,
        no_repeat_ngram_size=3,
        pad_token_id=tok.eos_token_id,
    )
print(tok.decode(out[0], skip_special_tokens=True))

adapter_dir = Path(OUT_DIR) / "adapter"
token_dir   = Path(OUT_DIR) / "tokenizer"
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)


**Noise filter**, the irony

In [14]:
def wakefy_many(prompt, k=8, max_new_tokens=90, temperature=1.1, top_p=0.92, rep=1.08, seed=None):
    if seed is not None:
        torch.manual_seed(seed);
        if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True, temperature=temperature, top_p=top_p, top_k=0,
            repetition_penalty=rep, no_repeat_ngram_size=3,
            num_return_sequences=k, pad_token_id=tok.eos_token_id, eos_token_id=tok.eos_token_id
        )
    texts = [tok.decode(o, skip_special_tokens=True) for o in outs]
    return texts


In [15]:
import re
VOWELS = set("aeiouyáéíóúàèìòùäëïöüâêîôûAEIOUY")
STEMS = set("""
river run eve adam milk city shop quick fast slow old new love night day dream wake
livre livre- livre's livrement livreur coeur coeur- coeur's coeurage coeurful
amor casa latte pane latte di pane latteo
wasser nacht stern haus liebe
éirinn dublin gael sean mór beag
""".split())

def has_vowel(word):
    return any(ch in VOWELS for ch in word)

def consonant_max_run(word):
    runs = re.findall(r"[^aeiouyAEIOUY]+", word)
    return max((len(r) for r in runs), default=0)

def count_stems(text):
    # crude: stem presence via substring overlap
    t = text.lower()
    return sum(1 for s in STEMS if s and s in t)

def joyce_score(text):
    # strip prompt header if present
    if "Wakefied:" in text: text = text.split("Wakefied:",1)[-1]
    words = re.findall(r"[A-Za-z’'-]+", text)
    if not words: return -1e9

    vowels_ok = sum(has_vowel(w) for w in words)/len(words)
    mean_len  = sum(len(w) for w in words)/len(words)
    long_ugly = sum(1 for w in words if len(w)>20 or consonant_max_run(w)>5)
    caps_weird= sum(1 for w in words if re.fullmatch(r"[A-Z][a-z]+[A-Z]$", w) or w.isupper())

    stem_hits = count_stems(text)

    # score: reward vowel-bearing, stems, moderate length; penalize ugly shards
    score = (
        2.0*vowels_ok
        + 0.4*mean_len
        + 1.5*stem_hits
        - 1.2*long_ugly
        - 0.8*caps_weird
    )
    return score

def wakefy_best(plain, k=12, **gen_kwargs):
    prompt = f"{SYSTEM}\n\nPlain: {plain}\nWakefied:"
    cand = wakefy_many(prompt, k=k, **gen_kwargs)
    scored = sorted(cand, key=joyce_score, reverse=True)
    return scored[0], [(joyce_score(c), c) for c in scored[:5]]


i'm sorry this is messy ...

In [ ]:
import torch
from transformers import TextStreamer

def fw_sample(
    model, tokenizer,
    prompt="riverrun, past Eve and Adam’s, from swerve of shore to bend of bay,",
    max_new_tokens=300,
    temperature=1.35,      # >1
    top_p=0.92,            # nucleus
    top_k=0,               # let nucleus do the work
    repetition_penalty=1.1,# discourages boring loops
    no_repeat_ngram_size=0,# Joycean recursion is fine for now
    do_sample=True,
):
    model.eval()
    device = next(model.parameters()).device
    ids = tokenizer(prompt, return_tensors="pt").to(device)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    with torch.no_grad():
        out = model.generate(
            **ids,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            eos_token_id=None,
            pad_token_id=tokenizer.eos_token_id,
            streamer=streamer,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(fw_sample(model, tokenizer))


In [ ]:
def post_clean(s):
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r"(?<!\.\s)(?<!^)([A-Z][a-z]+[A-Z])(?![A-Za-z])", lambda m: m.group(1).lower(), s)
    s = re.sub(r"([A-Za-z’'-]{18,})", r"\1\u00AD", s)
    return s

In [ ]:
def flag_fp_bits(text):
    toks = re.findall(r"[A-Za-z’'-]+", text)
    flags = []
    for w in toks:
        if not has_vowel(w): flags.append(("no_vowel", w))
        if consonant_max_run(w) >= 6: flags.append(("consonant_run≥6", w))
        if re.search(r"[A-Z]$", w): flags.append(("trailing_cap", w))
        if len(w) > 20: flags.append(("very_long", w))
    return flags

At the representational core sits TinyLlama-1.1B-Chat, deployed under mixed-precision arithmetic (bf16/fp16) to optimize throughput on limited-memory T4 hardware. The model’s internal cache is disabled (use_cache=False), and gradient checkpointing is invoked to trade recomputation for reduced memory allocation. The resultant architecture acts as a self-attenuating transformer: it “forgets” in order to remember within bounds, mirroring the cyclical amnesia of the Wakean text.

The system applies Low-Rank Adaptation (LoRA) across attention and MLP projection submodules (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj) thereby constraining fine-tuning to a low-dimensional manifold within parameter space. This selective plasticity produces what might be termed controlled stylistic drift: the model learns Joyce’s texture rather than his lexicon. The adapters’ hyperparameters (r=16, α=32, dropout=0.05) instantiate a medium-temperature update regime balancing expressivity and stability. Conceptually, LoRA functions here as a poetic prosthesis, a syntactic exoskeleton grafted onto the pretrained linguistic body.

Due to the GPU’s constrained VRAM capacity, each forward–backward pass processes a micro-batch of size 1, while gradients are accumulated over 32 iterations before a single optimization step. This simulates a virtual batch of 32 without exceeding hardware limits. Technically, this implements delayed gradient aggregation; rhetorically, it allegorizes the model’s incremental cognition: many small thoughts precipitating one grand revelation. The accumulation mechanism thus becomes both a computational strategy and a metaphor for compositional patience.

Training operates within an extended epochic horizon (epochs=1 but looping across numerous windows) using a cosine-annealed learning rate (lr=1e-4, warmup_ratio=0.03). Frequent checkpointing establishes a temporal ecology of model states, allowing resumption under volatile runtime constraints. The “save-steps” routine externalizes the process’s fragmentary nature (mirroring textual serialization in Work in Progress) by preserving multiple partial selves of the model as artifacts of its ongoing metamorphosis.

Inference employs a stochastic sampling configuration (temperature = 0.9, top_p = 0.95) optimized for stylistic exuberance rather than lexical precision. The generation prompt functions as both benchmark and incantation, a recursive self-reference to the corpus origin. Output decoding omits special tokens, yielding the model’s pure linguistic effervescence.

Technically a fine-tuned transformer; conceptually, a computational poetics experiment exploring how ml architectures can approximate literary un-readability. The Wakefyer thus redefines “loss” not as degradation but as aesthetic principle: every rounding error, every truncated sequence, every low-rank projection is a microcosm of Joyce’s deliberate semantic noise. The pipeline enacts a theory of productive distortion, situating training itself as a form of creative misprision.

to be continued